<a href="https://colab.research.google.com/github/albert-yue/gcn-explanability/blob/master/gcn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd gdrive/My\ Drive/gcn_explainability

/content/gdrive/My Drive/gcn_explainability


In [0]:
! git commit -m "Update"

[master 6d4a833] Update
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite gcn.ipynb (97%)


In [0]:
! git pull

Auto-merging gcn.ipynb
CONFLICT (content): Merge conflict in gcn.ipynb
Automatic merge failed; fix conflicts and then commit the result.


In [0]:
%ls

data/  gcn.ipynb  logs.pt  notebooks/  README.md  src/


In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
from src.models.gcn import GCN
import torch
import numpy as np
import scipy.sparse as sparse
import scipy
from src.data import Corpus, get_data, get_labels, get_vocabulary
from src.utils import load_sparse_tensor
import torch.nn as nn
from torch.autograd import Variable
from src.preprocessing import clean_text, build_adj_matrix, normalize_adj

logs = (200, 0.5, 41664, 23, 'logs.pt')
r8 = (200, 0.5, 13865, 8, 'gcn_r8_full.pt', 6191, 7674, 5485, 2189, 'r8')
r52 = (200, 0.5, 13865, 8, 'gcn_r52_full.pt', 6191, 7674, 5485, 2189, 'r52')

hidden_size = 200
dropout = 0.5

name_label = 'mr'
path = 'gcn_' + name_label + '_full.pt'
vocabulary = get_vocabulary('data/'+name_label+'-vocabulary.txt')
labels = get_labels('data/'+name_label+'-labels.txt')
test_corpus = get_data('data/test-'+name_label+'.txt', labels)
train_corpus = get_data('data/train-'+name_label+'.txt', labels)

clean_text(test_corpus, vocabulary)
clean_text(train_corpus, vocabulary)

num_words = len(vocabulary)
training_docs = len(train_corpus)
test_docs = len(test_corpus)
num_docs = training_docs + test_docs
num_vertices = num_words + num_docs
num_labels = len(labels)
print(num_vertices)


29248


In [0]:
def matrices(path, adj, index, label):
  model = GCN(num_vertices, hidden_size, num_labels, dropout=dropout)
  model.load_state_dict(torch.load(path))
  model.eval()
  loss_fn = nn.CrossEntropyLoss()
  model.zero_grad()
  output = model(adj)
  loss = loss_fn(output[-index:-index+1,:], label)
  loss.retain_grad()
  loss.backward()
  weight_matrices = []
  gradients = []
  for n, p in model.named_parameters():
    print(n)
    weight_matrices.append(p.detach().numpy())
    gradients.append(p.grad.numpy())
    print(p.grad.shape)
  return weight_matrices, gradients, output

In [0]:
def grad_cam_avg(adj, weight_matrices, gradients):
  adj = adj.coalesce()
  V = sparse.csr_matrix((adj.values(), adj.indices()), shape=list(adj.size()))
  F_1 = scipy.special.expit(V.dot(weight_matrices[0]))
  F_2 = scipy.special.expit(V.dot(np.matmul(F_1, weight_matrices[1])))
  F = [F_1, F_2]
  grad_cams = []
  for l in range(2):
    grad_cam = []
    grad_cams.append(grad_cam)
    alphas = np.mean(gradients[l], axis = 0)
    for n in range(num_vertices):
      grad_cam.append(np.maximum(0, np.dot(alphas, F[l][n, :])))

  grad_cam_avgs = np.mean(grad_cams, axis=0)
  return grad_cams, grad_cam_avgs

In [0]:
def convert_to_words(top_words, vocabulary, labels, train_corpus):
  words = []
  for word_idx in top_words:
      if word_idx < num_words:
        words.append("word: " + vocabulary[word_idx])
      elif word_idx < num_words+training_docs:
        words.append(labels[train_corpus[word_idx-num_words].label])
      else:
        words.append(labels[test_corpus[word_idx-num_words-training_docs].label])
  return words

In [0]:
def p_carrot_val(carrot, p, weight_matrix):
  p_carrot = np.matmul(np.maximum(0, weight_matrix), carrot.T)
  p_carrot = p_carrot / np.sum(p_carrot, axis= 0)
  p_carrot = np.matmul(p_carrot, p)
  return p_carrot

def p_F_val(v, F, p_carrot):
  p_F = v.dot(F)
  p_F = p_F / np.sum(p_F, axis= 0)
  p_F = np.matmul(p_F, p_carrot.T)
  return p_F

def EB(adj, weight_matrices, gradients, output):
  adj = adj.coalesce()
  V = sparse.csr_matrix((adj.values(), adj.indices()), shape=list(adj.size()))
  F_1 = scipy.special.expit(V.dot(weight_matrices[0]))
  F_2 = scipy.special.expit(V.dot(np.matmul(F_1, weight_matrices[1])))
  F = [F_1, F_2]
  F_carrot = [V.dot(F_1), V.dot(F_2)]
  p_F3 = output.detach().numpy()
  p_carrot_F2 = p_carrot_val(F_carrot[1], p_F3, weight_matrices[1])
  p_F2 = p_F_val(V, F[1], p_carrot_F2)
  p_carrot_F1 = p_carrot_val(F_carrot[0], p_F2, weight_matrices[0])
  p_F1 = p_F_val(V, F[0], p_carrot_F1)
  p_F = [p_F1, p_F2, p_F3]
  p_carrot_F = [p_carrot_F1, p_carrot_F2]
  eb_vals = np.mean(p_F1, axis=1)
  return eb_vals

  


In [23]:

#grad_cam for specific docs
indices = [5, 6]
for i in indices:
  single_corpus = Corpus([test_corpus.data[-i]])
  adj = build_adj_matrix(single_corpus, vocabulary, num_docs, training_docs+i)
  adj = normalize_adj(adj)
  weight_matrices, gradients, output = matrices(path, adj, i, single_corpus.labels())
  grad_cams, grad_cam_avgs = grad_cam_avg(adj, weight_matrices, gradients)
  print(test_corpus.data[-i].text)
  print(labels[test_corpus.data[-i].label])
  print(labels[np.argmax(output[-i,:].detach().numpy())])
  for name, grad_cam_vals in [('layer1', grad_cams[0]), ('layer2', grad_cams[1]), ('overall', grad_cam_avgs)]:
    print(name+":\n")
    top_words_idx = np.asarray(grad_cam_vals).argsort()[-10:]
    top_words = convert_to_words(reversed(top_words_idx), vocabulary, labels, train_corpus)
    for word in top_words:
      print(word)
  
  max_grads = np.asarray(np.linalg.norm(np.matmul(gradients[0], gradients[1]),axis=1)).argsort()[-10:]
  print(convert_to_words(reversed(max_grads), vocabulary, labels, train_corpus))

  if num_vertices < 50000:
    eb_vals = EB(adj, weight_matrices, gradients, output)
    top_words_idx = np.asarray(eb_vals).argsort()[-10:]
    top_words = convert_to_words(reversed(top_words_idx), vocabulary, labels, train_corpus)
    for word in top_words:
      print(word)



Building word frequencies per doc



Building word frequencies per window



Calculating PMIs



Calculating TF-IDF



Identities


...


Building word frequencies per window



Calculating PMIs



Calculating TF-IDF



Identities



layer1.weight
torch.Size([29248, 200])
layer2.weight
torch.Size([200, 2])
['tour', 'de', 'force', 'modern', 'cinema']
1
1
layer1:

1
word: callar
word: absorbs
word: hallmarks
word: insatiable
word: olds
word: unthinkable
word: posturing
word: strutting
word: p
layer2:

1
word: callar
word: absorbs
word: hallmarks
word: insatiable
word: olds
word: unthinkable
word: posturing
word: strutting
word: p
overall:

1
word: callar
word: absorbs
word: hallmarks
word: insatiable
word: olds
word: unthinkable
word: posturing
word: strutting
word: p
['1', '1', 'word: decidiram', 'word: absorbs', 'word: hallmarks', 'word: insatiable', 'word: olds', 'word: unthinkable', 'word: posturing', 'word: strutting']
word: de
word: tour
word: modern
word: force
word: cinema
word: rehearsal
1
0
1
word: brawn


In [36]:


total = 0
max_num = 10
he = test_docs -10
for i in range(1, he):
  if test_corpus.data[-i].label == np.argmax(output[-i,:].detach().numpy()):
    print(i)
    total += 1
    if total == max_num:
      break

total = 0
for i in range(1, he):
  if test_corpus.data[-i].label == np.argmax(output[-i,:].detach().numpy()):
    total += 1
print(total/he)

total = 0
for i in range(he, 1, -1):
  if test_corpus.data[-i].label != np.argmax(output[-i,:].detach().numpy()):
    print(i)
    total += 1
    if total == max_num:
      break

1
2
3
4
5
6
7
10
11
12
0.8063331803579624
2141
2140
2135
2130
2119
2114
2111
2110
2104
2095
